In [ ]:
# Number of Early Adopters by Year
#
# NOTE limitation on q.year to speed test run time.
# Eventually will need to write to CSV and process in batches. 
#
# # 5316210 total db hits in 6233 ms

try_next_year = get_last_year_completed("/tmp/data/result/FeatureExtractionResults/EarlyAdopters/")+1
try_next_year = 1950 # included to boostrap at 2000

for i in tqdm(range(try_next_year, 2018)):
    print(i)
    query = """
    MATCH (a:Author)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta {venue:"Nature"})
    WHERE p.year < q.year + 4 AND q.year = """+str(i)+""" 
    WITH *, q.year+1 as p1, q.year+2 as p2, q.year+3 as p3
    WITH q.title as title, p1, p2, p3, q.year as qyear, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name]) AS year_3
    return 
        title, 
        size(year_1) as early_adopters_1, 
        size(year_2) as early_adopters_2,
        size(year_3) as early_adopters_3"""

    df_earlyadopters = query_to_df(query, graph)
    df_earlyadopters.to_csv('/tmp/data/result/FeatureExtractionResults/EarlyAdopters/early_adopters_'+str(i)+'.csv', index=False, columns = ['title', 'early_adopters_1', 'early_adopters_2', 'early_adopters_3'])
    

In [ ]:
# FIRST REMAKE
# 5316210 total db hits in 5904 ms

MATCH (a:Author)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
WHERE p.year < q.year + 4 AND q.year = 2010 
WITH q.title as title, q.year+1 as p1, q.year+2 as p2, q.year+3 as p3, q.year as qyear, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2
RETURN 
    title,
    SIZE(year_1) as early_adopters_1,
    SIZE(year_2) as early_adopters_2,
    SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# FIRST REMAKE BUT WITH INBUILT P1-P3
# 4426632 total db hits in 6245 ms

MATCH (a:Author)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
WHERE p.year < q.year + 4 AND q.year = 2010 
WITH q.title as title, 2011 as p1, 2012 as p2, 2013 as p3, q.year as qyear, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2
RETURN 
    title,
    SIZE(year_1) as early_adopters_1,
    SIZE(year_2) as early_adopters_2,
    SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# SECOND REMAKE
# 4002410 total db hits in 10511 ms then 4055 ms

MATCH (q:Quanta)
WHERE q.year = 2010
WITH q
MATCH (p:Quanta)-[:CITES]->(q:Quanta)
WHERE p.year < q.year + 4
WITH DISTINCT p, q.title as title
MATCH (a:Author)-[:AUTHORED]->(p)
WITH title, 2011 as p1, 2012 as p2, 2013 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2
RETURN title, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
#BEST VERSION

PROFILE MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y:Year {value:2009})
USING INDEX y:Year(value) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2 
RETURN title, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
PROFILE MATCH (y:Year {value:2009})
USING INDEX y:Year(value) 
MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2 
RETURN title, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# THIRD REMAKE
# 4002410 total db hits in 73769 ms then 5602 ms

MATCH (p:Quanta)-[:CITES]->(q:Quanta)
WHERE q.year = 2010 and p.year < q.year + 4
WITH DISTINCT p, q.title as title
MATCH (a:Author)-[:AUTHORED]->(p)
WITH title, 2011 as p1, 2012 as p2, 2013 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_1 | x.name]) AS year_2
RETURN title, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# INCLUDING YEAR OF PUBLICATION

MATCH (p:Quanta)-[:CITES]->(q:Quanta)<-[:PUBLISHED_IN_VENUE]-(v:Venue)
WHERE q.year = 2000 and p.year < q.year + 4 
WITH DISTINCT p, q.title as title
MATCH (a:Author)-[:AUTHORED]->(p)
WITH title, 2000 as p0, 2001 as p1, 2002 as p2, 2003 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p0 | x.name]) AS year_0
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 and not x.name in year_0 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_0 and not x.name in year_1 | x.name]) AS year_2
RETURN title, size(year_0) as early_adopters_0, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_0 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# EXTENDED 

PROFILE MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y:Year {value:2009})
USING INDEX y:Year(value) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p0 | x.name]) AS year_0
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p1 and not x.name in year_0 | x.name]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in year_0 and not x.name in year_1 | x.name]) AS year_2
RETURN title, size(year_0) as early_adopters_0, size(year_1) as early_adopters_1, size(year_2) as early_adopters_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_0 and not x.name in year_1 and not x.name in year_2 | x.name])) AS early_adopters_3

In [ ]:
# TOP 50 PAPERS

MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE q.year = 2018 and p.year >= q.year 
WITH DISTINCT p, q.title as cited_paper ORDER BY p.year LIMIT 50
UNWIND apoc.coll.sortMulti(collect({title:p.title, year:p.year}), ['^year'], 50) AS adopting_papers 
RETURN cited_paper, collect(adopting_papers.title) as early_adopting_papers

In [ ]:
# POSSIBLY FASTER PAPERS
# This one ran because the collect statement was placed ahead of the sortMulti

MATCH (p:Quanta)-[:CITES]->(q:Quanta)
WHERE q.year = 2018 and p.year >= q.year
WITH DISTINCT q.title as cited_paper, collect({title:p.title, year:p.year}) as adopting_collection ORDER BY p.year LIMIT 50
UNWIND apoc.coll.sortMulti(adopting_collection, ['^year'], 50) AS early_adopting_papers
RETURN cited_paper, collect(early_adopting_papers.title) as early_adopting_papers_titles

In [ ]:
# TOP 50 AUTHORS

MATCH (p:Quanta)-[:CITES]->(q:Quanta)
WHERE p.year > q.year
WITH DISTINCT p, q.title as cited_paper ORDER BY p.year LIMIT 50
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH a.name as author_name, cited_paper, p.year as paper_year
WITH cited_paper, apoc.coll.toSet(collect({name:author_name, year:paper_year})) as author_mapping
UNWIND apoc.coll.sortMulti(author_mapping, ['year'], 50) AS author_list
RETURN cited_paper, collect(author_list.name) AS top_adopters

# Not Implementing Stuff Below

In [1]:
def find_cited(i):
    """
    MATCH (a:Author)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta {venue:"Nature"})
    WHERE p.year < q.year + 4 AND q.year = """+str(i)+"""
    RETURN q.year+1 as p1, q.year+2 as p2, q.year+3 as p3, q.title as title, q.year as qyear, 
    apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist, 
    apoc.coll.toSet(collect({paper:p, community:a.labelprop})) as clist
    """

In [ ]:
def find_early_adopters_1(p1, alist):
    y1 = apoc.coll.toSet([x IN alist WHERE x.year=p1 | x.name])
    return y1

def find_early_adopters_2(p2, alist, y1):
    y2 = apoc.coll.toSet([x IN alist WHERE x.year=p2 and not x.name in y1 | x.name])
    return y2

def find_early_adopters_3(p3, alist, y1, y2):
    y3 = apoc.coll.toSet([x IN alist WHERE x.year=p3 and not x.name in year_1 and not x.name in year_2 | x.name])
    return y3

year_1 = find_early_adopters_1(p1, alist)
year_2 = find_early_adopters_2(p2, alist, year_1)
year_3 = find_early_adopters_3(p3, alist, year_1, year_2)

print ('early_adopters_1: ' + len(year_1), 'early_adopters_2: ' + len(year_2), 'early_adopters_3: ' + len(year_3))

# Back to Things That May Be Implemented

In [ ]:
# Number of Infected Communities
#
# NOTE limitation on q.year to speed test run time.
# Eventually will need to write to CSV and process in batches. 
#

base_dir = '/tmp/data/result/FeatureExtractionResults/'
data_dir = base_dir + 'InfectedCommunities/'

try_next_year = get_last_year_completed(data_dir)+1
for i in tqdm(range(try_next_year, 2018)):
#     for j in range(1,4):
#         query = """
#         MATCH (a:NatureAuthor)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
#         WHERE p.year={}+{} AND q.year={}
#         RETURN q.title as title, size(apoc.coll.toSet(collect(a.labelprop)))
#         """.format(i,j,i)
#         df_infectedcommunities = query_to_df(query, graph)
#         df_infectedcommunities.to_csv(data_dir + 'infected_communities_{}_{}.csv'.format(i,j), 
#                                       index=False, 
#                                       columns = ['title', 'infected_communities'])
    
    #df_infectedcommunities.head()
    query = """
    MATCH (a:NatureAuthor)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
    WHERE p.year < q.year + 4 AND q.year = """+str(i)+""" 
    WITH *, q.year+1 as p1, q.year+2 as p2, q.year+3 as p3
    WITH q.title as title, p1, p2, p3, q.year as qyear, apoc.coll.toSet(collect({person:a, year:p.year})) AS alist
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person.labelprop]) AS year_1
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person.labelprop]) AS year_2
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person.labelprop]) AS year_3
    RETURN 
        title, 
        size(year_1) as infected_communities_1, 
        size(year_2) as infected_communities_2, 
        size(year_3) as infected_communities_3"""

    df_infectedcommunities = query_to_df(query, graph)
    df_infectedcommunities.to_csv('/tmp/data/result/FeatureExtractionResults/InfectedCommunities/infected_communities_'+str(i)+'.csv', index=False, columns = ['title', 'infected_communities_1', 'infected_communities_2', 'infected_communities_3'])
    #df_infectedcommunities.head()

In [ ]:
PROFILE MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y:Year {value:2009})
USING INDEX y:Year(value) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person.labelprop]) AS year_1
WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person.labelprop]) AS year_2
RETURN title, size(year_1) as infected_communities_1, size(year_2) as infected_communities_2, SIZE(apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person.labelprop])) AS infected_communities_3

In [ ]:
# IGNORE FOR NOW

def find_infected(alist, p1, p2, p3):
    y1 = apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person.labelprop])
    y2 = apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person.labelprop])
    y3 = apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person.labelprop])
    return [y1, y2, y3]

infected = find_year_1_infected(alist, p1, p2, p3)
print('infected communities 1: ' + infected[0], 'infected communities 2: ' + infected[1], 'infected communities 3: ' + infected[2])

In [ ]:
# Usage Entropy
#
# NOTE limitation on q.year to speed test run time.
# Eventually will need to write to CSV and process in batches. 
#
try_next_year = get_last_year_completed("/tmp/data/result/FeatureExtractionResults/UsageEntropy/")+1

for i in tqdm(range(try_next_year, 2018)):
    query = """
    MATCH (a:NatureAuthor)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
    WHERE p.year < q.year + 4 AND q.year= """+str(i)+"""
    WITH 
        q, q.title as title, 
        apoc.coll.toSet(collect({paper:p, community:a.labelprop})) as clist, 
        q.year+1 as p1, q.year+2 as p2, q.year+3 as p3
    WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p1|x.community]) AS year_1_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p1|x.paper])) as s1
    WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p2|x.community]) AS year_2_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p2|x.paper])) as s2
    WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p3|x.community]) AS year_3_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p3|x.paper])) as s3
    RETURN 
        title, 
        reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2)) as usage_entropy_1,
        reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2)) as usage_entropy_2,
        reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2)) as usage_entropy_3"""

    df_usageentropy = query_to_df(query, graph)
    df_usageentropy.to_csv('/tmp/data/result/FeatureExtractionResults/UsageEntropy/usage_entropy_'+str(i)+'.csv', index=False, columns = ['title', 'usage_entropy_1', 'usage_entropy_2', 'usage_entropy_3'])
    #df_usageentropy.head()

In [ ]:
PROFILE MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y:Year {value:2009})
USING INDEX y:Year(value) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({paper:p, community:a.labelprop})) as clist
WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p1|x.community]) AS year_1_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p1|x.paper])) as s1
WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p2|x.community]) AS year_2_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p2|x.paper])) as s2
WITH *, apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p3|x.community]) AS year_3_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p3|x.paper])) as s3
RETURN 
        title, 
        reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2)) as usage_entropy_1,
        reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2)) as usage_entropy_2,
        reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2)) as usage_entropy_3

In [ ]:
def find_usage_entropy(clist, p1, p2, p3):
    ue1 = apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p1|x.community]) AS year_1_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p1|x.paper]))
    ue2 = apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p2|x.community]) AS year_2_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p2|x.paper]))
    ue3 = apoc.coll.frequencies([x IN clist WHERE x.paper.year<=p3|x.community]) AS year_3_count, size(apoc.coll.toSet([x IN clist WHERE x.paper.year<=p3|x.paper]))
    reduced_ue1 = reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2))
    reduced_ue2 = reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2))
    reduced_ue3 = reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2))
    return [reduced_ue1, reduced_ue2, reduced_ue3]

usage_entropy = find_usage_entropy(clist, p1, p2, p3)
print('usage_entropy_1: ' + usage_entropy[0], 'usage_entropy_2: ' + usage_entropy[1], 'usage_entropy_3: ' + usage_entropy[2])

In [ ]:
# Adoption Entropy
#
# NOTE limitation on q.year to speed test run time.
# Eventually will need to write to CSV and process in batches. 
#
try_next_year = get_last_year_completed("/tmp/data/result/FeatureExtractionResults/AdoptionEntropy/")+1

for i in tqdm(range(try_next_year, 2018)):
    query = """
    MATCH (a:NatureAuthor)-[:AUTHORED]->(p:Quanta)-[:CITES]->(q:Quanta)
    WHERE p.year < q.year + 4 AND q.year= """+str(i)+"""
    WITH *, q.year+1 as p1, q.year+2 as p2, q.year+3 as p3
    WITH q.title as title, p1, p2, p3, 
        apoc.coll.toSet(collect({person:a, year:p.year})) AS alist
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person]) AS year_1_people
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person]) AS year_2_people
    WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person]) AS year_3_people
    WITH *, apoc.coll.frequencies([x IN year_1_people | x.labelprop]) AS year_1_count, size(year_1_people) as s1
    WITH *, apoc.coll.frequencies([x IN year_2_people | x.labelprop]) AS year_2_count, size(year_2_people) as s2
    WITH *, apoc.coll.frequencies([x IN year_3_people | x.labelprop]) AS year_3_count, size(year_3_people) as s3
    RETURN 
        title, 
        reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2)) as adoption_entropy_1,
        reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2)) as adoption_entropy_2, 
        reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2)) as adoption_entropy_3"""

    df_adoptionentropy = query_to_df(query, graph)
    df_adoptionentropy.to_csv('/tmp/data/result/FeatureExtractionResults/AdoptionEntropy/adoption_entropy_'+str(i)+'.csv', index=False, columns = ['title', 'adoption_entropy_1', 'adoption_entropy_2', 'adoption_entropy_3'])
    #df_adoptionentropy.head()

In [ ]:
PROFILE MATCH (q:Quanta)-[:PUBLISHED_IN_YEAR]->(y:Year {value:2009})
USING INDEX y:Year(value) 
WITH DISTINCT q, y.value as year 
MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
WHERE p.year < q.year + 4 
WITH DISTINCT p, q.title as title, year 
MATCH (a:Author)-[:AUTHORED]->(p) 
WITH title, year+1 as p1, year+2 as p2, year+3 as p3, apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist 
WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person]) AS year_1_people
WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person]) AS year_2_people
WITH *, apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person]) AS year_3_people
WITH *, apoc.coll.frequencies([x IN year_1_people | x.labelprop]) AS year_1_count, size(year_1_people) as s1
WITH *, apoc.coll.frequencies([x IN year_2_people | x.labelprop]) AS year_2_count, size(year_2_people) as s2
WITH *, apoc.coll.frequencies([x IN year_3_people | x.labelprop]) AS year_3_count, size(year_3_people) as s3
RETURN 
    title, 
    reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2)) as adoption_entropy_1,
    reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2)) as adoption_entropy_2, 
    reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2)) as adoption_entropy_3

In [ ]:
def find_adoption_energy(alist, p1, p2, p3):
    y1_people = apoc.coll.toSet([x IN alist WHERE x.year<=p1|x.person])
    y2_people = apoc.coll.toSet([x IN alist WHERE x.year<=p2|x.person])
    y3_people = apoc.coll.toSet([x IN alist WHERE x.year<=p3|x.person])
    y1_count = apoc.coll.frequencies([x IN y1_people | x.labelprop])
    s1 = len(y1_people)
    y2_count = apoc.coll.frequencies([x IN y2_people | x.labelprop])
    s2 = len(y2_people)
    y3_count = apoc.coll.frequencies([x IN y3_people | x.labelprop])
    s3 = len(y3_people) 
    reduced_y1 = reduce(i = 0.0, x IN year_1_count| i - toFloat(x.count)/s1*log(toFloat(x.count)/s1)/log(2))
    reduced_y2 = reduce(i = 0.0, x IN year_2_count| i - toFloat(x.count)/s2*log(toFloat(x.count)/s2)/log(2))
    reduced_y3 = reduce(i = 0.0, x IN year_3_count| i - toFloat(x.count)/s3*log(toFloat(x.count)/s3)/log(2))
    return [reduced_y1, reduced_y2, reduced_y3]

adoption_energy = find_adoption_energy(alist, p1, p2, p3)
print('adoption energy 1: ' + adoption_energy[0], 'adoption energy 2: ' + adoption_energy[1], 'adoption energy 3: ' + adoption_energy[2])

In [ ]:
def find_all_information():
    '''
    for i in tqdm(range(try_next_year, 2018)):
        CALL find_cited(i)
        YIELD q.year+1 as p1, q.year+2 as p2, q.year+3 as p3, q.title as title, q.year as qyear, 
        apoc.coll.toSet(collect({name:a.name, year:p.year})) AS alist, 
        apoc.coll.toSet(collect({paper:p, community:a.labelprop})) as clist
        
        year_1_early_adopters = find_early_adopters_1(p1, alist)
        year_2_early_adopters = find_early_adopters_2(p2, alist, year_1)
        year_3_early_adopters = find_early_adopters_3(p3, alist, year_1, year_2)

print ('early_adopters_1: ' + len(year_1), 'early_adopters_2: ' + len(year_2), 'early_adopters_3: ' + len(year_3))
    '''